<a href="https://colab.research.google.com/github/ANKSAditya/Colab_AIML/blob/main/Cloud_local_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***WARNING***

> This command can delete entire disk.

> Run this programma at your on risk.

> If you want run this programma then copy/paste

import shutil
import os

import os
import shutil
import subprocess
import psutil
import time

# --- Utility Functions ---
def get_disk_usage(path="/"):
    usage = shutil.disk_usage(path)
    return {
        "total": usage.total / (1024**3),
        "used": usage.used / (1024**3),
        "free": usage.free / (1024**3)
    }

def humanize_size(size_gb):
    return f"{size_gb:.2f} GB"

def colored(text, color_code):
    return f"\033[{color_code}m{text}\033[0m"

def print_status(message, status="info"):
    colors = {"info": "36", "success": "32", "error": "31", "warn": "33"}
    print(colored(message, colors.get(status, "36")))

# --- Cleanup Function ---
def clean_cache(confirm=True):
    cache_dirs = [
        "/root/.cache/huggingface",
        "/root/.cache/torch",
        "/root/.cache/transformers",
        "/root/.cache",
        "/root/.npm",   # Node cache
        "/root/.nv",    # NVIDIA cache
    ]

    print_status("📊 Checking disk usage before cleanup...", "info")
    before = get_disk_usage("/")
    print_status(f"Total: {humanize_size(before['total'])} | Used: {humanize_size(before['used'])} | Free: {humanize_size(before['free'])}", "info")
    print()

    if confirm:
        user_input = input("⚠️ Do you really want to delete all cache files? (yes/no): ").strip().lower()
        if user_input != "yes":
            print_status("❌ Cleanup aborted by user.", "warn")
            return

    freed_space = 0
    for cache_dir in cache_dirs:
        if os.path.exists(cache_dir):
            try:
                size_before = psutil.disk_usage(cache_dir).used if os.path.isdir(cache_dir) else 0
                shutil.rmtree(cache_dir, ignore_errors=True)
                print_status(f"✅ Deleted: {cache_dir}", "success")
            except Exception as e:
                print_status(f"⚠️ Could not delete {cache_dir}: {e}", "warn")
        else:
            print_status(f"❌ Not found: {cache_dir}", "error")

    # Extra: Clear pip cache
    try:
        subprocess.run(["pip", "cache", "purge"], check=True)
        print_status("🧹 pip cache cleared!", "success")
    except Exception as e:
        print_status(f"⚠️ Could not clear pip cache: {e}", "warn")

    print("\n📊 Checking disk usage after cleanup...")
    after = get_disk_usage("/")
    print_status(f"Total: {humanize_size(after['total'])} | Used: {humanize_size(after['used'])} | Free: {humanize_size(after['free'])}", "info")

    freed = after["free"] - before["free"]
    print_status(f"\n💾 Cleanup complete! Space freed: {humanize_size(freed)}", "success")

# --- Run Cleanup ---
clean_cache(confirm=True)

**Model precaution**

This is old AI modul and compatable with resource T4 GPU in colab.

Some answer will be correct according to old AI modul and some answer will be incorrect so please do not depend on this information.

GPU module

In [ ]:
!pip -q install --upgrade pip
# Install compatible versions
!pip install -q torch torchvision torchaudio  # Get latest Colab-compatible torch
!pip install -q transformers==4.41.2 accelerate==0.25.0 sentencepiece


import os, gc
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def printt(*a):
    print(*a)

# 2) Device detection: CUDA -> CPU
def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        backend = "CUDA"
    else:
        device = torch.device("cpu")
        backend = "CPU"
    printt(f"Backend: {backend} | Device: {device}")
    return backend, device

backend, device = get_device()

# 3) Model loading with float16 for GPU and float32 fallback for CPU
PRIMARY_MODEL = "google/flan-t5-xl"
FALLBACK_MODEL = "google/flan-t5-base"

def try_load_model(model_name, device, use_fp16=True):
    if backend == "CUDA" and use_fp16:
        dtype = torch.float16
    else:
        dtype = torch.float32

    printt(f"Loading model {model_name} with dtype={dtype} ...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=dtype)
    model.to(device)
    printt("Loaded:", model_name)
    return tokenizer, model

def safe_load(primary=PRIMARY_MODEL, fallback=FALLBACK_MODEL):
    try:
        return try_load_model(primary, device, use_fp16=True) + (primary,)
    except Exception as e:
        printt("⚠️ Primary model load failed:", e)
        gc.collect()
        if backend == "CUDA":
            torch.cuda.empty_cache()
        try:
            return try_load_model(fallback, device, use_fp16=False) + (fallback,)
        except Exception as e2:
            printt("⚠️ Fallback model load failed:", e2)
            raise RuntimeError("Could not load either primary or fallback model.")

tokenizer, model, active_model = safe_load()
printt("Active model:", active_model)

# 4) Prompt and batch generation helpers

def t5_prompt(question, short=False):
    if short:
        return f"Answer in one word.\nQuestion: {question}\nAnswer:"
    return f"Question: {question}\nAnswer:"

@torch.no_grad()
def generate_answer(question, short=True, max_new_tokens=16, num_beams=4):
    prompt = t5_prompt(question, short=short)
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=num_beams,
        early_stopping=True,
        use_cache=True
    )
    outputs = model.generate(**inputs, **gen_kwargs)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return text

# 5) Batch generation

@torch.no_grad()
def generate_batch(questions, short=True, max_new_tokens=16, num_beams=4):
    prompts = [t5_prompt(q, short=short) for q in questions]
    inputs = tokenizer(prompts, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=num_beams,
        early_stopping=True,
        use_cache=True
    )
    outputs = model.generate(**inputs, **gen_kwargs)
    return [tokenizer.decode(out, skip_special_tokens=True).strip() for out in outputs]

# 6) Disk and memory usage (optional)
def show_usage():
    try:
        printt("\n=== Disk usage ===")
        os.system("df -h / | head -2")
    except Exception:
        pass

show_usage()

# 7) Example queries - single and batch
questions = [
    "How many states in India?"
]

printt("\n--- Answers ---")
for q in questions:
    try:
        short_ans = generate_answer(q, short=True, max_new_tokens=8, num_beams=4)
        full_ans  = generate_answer(q, short=False, max_new_tokens=64, num_beams=4)
        printt(f"Q: {q}")
        printt("A (short):", short_ans)
        printt("A (full): ", full_ans)
        printt("-" * 40)
    except Exception as ex:
        printt(f"Generation failed for question `{q}`:", ex)

printt("\n✅ Done. Active model:", active_model)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.8 MB/s eta 0:00:00
Backend: CUDA | Device: cuda
Loading model google/flan-t5-xl with dtype=torch.float16 ...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Loaded: google/flan-t5-xl
Active model: google/flan-t5-xl

=== Disk usage ===

--- Answers ---
Q: How many states in India?
A (short): 29
A (full):  29
----------------------------------------

✅ Done. Active model: google/flan-t5-xl


In [ ]:
# =========================================================
# Install Dependencies (Run this in Colab separately first)
# =========================================================
# !pip -q install --upgrade pip
# !pip install -q torch torchvision torchaudio
# !pip install -q transformers==4.41.2 accelerate==0.25.0 sentencepiece

# =========================================================
# Imports & Setup
# =========================================================
import os, gc, torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Utility print
def printt(*a):
    print(*a)

# Optional: Speed up CUDA matmul
if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True

# =========================================================
# Device Detection
# =========================================================
def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        backend = "CUDA"
    else:
        device = torch.device("cpu")
        backend = "CPU"
    printt(f"Backend: {backend} | Device: {device}")
    return backend, device

backend, device = get_device()

# =========================================================
# Model Names
# =========================================================
PRIMARY_MODEL = "google/flan-t5-xl"
FALLBACK_MODEL = "google/flan-t5-base"
CACHE_DIR = "./models"  # Store models locally for faster re-runs

# =========================================================
# Model Loading
# =========================================================
def try_load_model(model_name, device, use_fp16=True):
    """
    Try loading a model with float16 for GPU and float32 fallback for CPU.
    Uses cache directory to avoid re-downloading.
    """
    dtype = torch.float16 if backend == "CUDA" and use_fp16 else torch.float32
    printt(f"Loading model {model_name} with dtype={dtype} ...")
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, cache_dir=CACHE_DIR)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_name,
        torch_dtype=dtype,
        cache_dir=CACHE_DIR,
        device_map="auto" if backend == "CUDA" else None
    )
    model.to(device)
    printt(f"✅ Loaded: {model_name}")
    return tokenizer, model

def safe_load(primary=PRIMARY_MODEL, fallback=FALLBACK_MODEL):
    """
    Attempts to load primary model, falls back if OOM or other error.
    """
    try:
        return try_load_model(primary, device, use_fp16=True) + (primary,)
    except Exception as e:
        printt(f"⚠️ Primary model load failed: {type(e).__name__}: {e}")
        gc.collect()
        if backend == "CUDA":
            torch.cuda.empty_cache()
        try:
            return try_load_model(fallback, device, use_fp16=False) + (fallback,)
        except Exception as e2:
            printt(f"❌ Fallback model also failed: {type(e2).__name__}: {e2}")
            raise RuntimeError("Could not load either primary or fallback model.")

tokenizer, model, active_model = safe_load()
printt(f"Active model: {active_model}")

# =========================================================
# Prompt Helper
# =========================================================
def t5_prompt(question, short=False):
    """
    Builds a prompt for T5 models.
    """
    if short:
        return f"Answer in one word.\nQuestion: {question}\nAnswer:"
    return f"Question: {question}\nAnswer:"

# =========================================================
# Answer Generation
# =========================================================
@torch.no_grad()
def generate_answer(question, short=True, max_new_tokens=16, num_beams=4):
    """
    Generate a single answer for a question.
    """
    prompt = t5_prompt(question, short=short)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=num_beams,
        early_stopping=True,
        use_cache=True
    )
    outputs = model.generate(**inputs, **gen_kwargs)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return text

# =========================================================
# Batch Generation
# =========================================================
@torch.no_grad()
def generate_batch(questions, short=True, max_new_tokens=16, num_beams=4):
    """
    Generate answers for a batch of questions.
    """
    prompts = [t5_prompt(q, short=short) for q in questions]
    inputs = tokenizer(prompts, return_tensors="pt", padding=True).to(device)

    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=num_beams,
        early_stopping=True,
        use_cache=True
    )
    outputs = model.generate(**inputs, **gen_kwargs)
    return [tokenizer.decode(out, skip_special_tokens=True).strip() for out in outputs]

# =========================================================
# Disk and Memory Usage
# =========================================================
def show_usage():
    """
    Shows disk usage and frees memory.
    """
    try:
        printt("\n=== Disk usage ===")
        os.system("df -h / | head -2")
    except Exception as e:
        printt(f"Disk usage check failed: {type(e).__name__}: {e}")
    gc.collect()
    if backend == "CUDA":
        torch.cuda.empty_cache()

show_usage()

# =========================================================
# Quick Model Test Function
# =========================================================
def test_model():
    """
    Quick sanity test for model loading.
    """
    try:
        q = "What is the capital of France?"
        printt("\n--- Quick Test ---")
        printt(f"Q: {q}")
        printt("A:", generate_answer(q, short=True))
    except Exception as ex:
        printt(f"Test failed: {type(ex).__name__}: {ex}")

# =========================================================
# Example Queries
# =========================================================
questions = [
    "where is india?"
]

printt("\n--- Answers ---")
for q in questions:
    try:
        short_ans = generate_answer(q, short=True, max_new_tokens=8, num_beams=4)
        full_ans  = generate_answer(q, short=False, max_new_tokens=64, num_beams=4)
        printt(f"Q: {q}")
        printt("A (short):", short_ans)
        printt("A (full): ", full_ans)
        printt("-" * 40)
    except Exception as ex:
        printt(f"Generation failed for `{q}`: {type(ex).__name__}: {ex}")

printt("\n✅ Done. Active model:", active_model)

# Run a quick test
test_model()


Backend: CUDA | Device: cuda
Loading model google/flan-t5-xl with dtype=torch.float16 ...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Loaded: google/flan-t5-xl
Active model: google/flan-t5-xl

=== Disk usage ===

--- Answers ---
Q: Write a hello in python
A (short): python
A (full):  if __name__ == '__main__': hello = ''.join([str(x) for x in input().split()])
----------------------------------------

✅ Done. Active model: google/flan-t5-xl

--- Quick Test ---
Q: What is the capital of France?
A: paris


In [ ]:
# ============================
# Colab-ready: FLAN-T5-XL (TPU stable)
# ============================

# 1) Install pinned, TPU-compatible libraries
# (This cell may take ~1-2 mins)
# ✅ Pin NumPy for TPU stability

!pip -q install --upgrade pip
!pip -q install torch==2.1.0
!pip -q install torch_xla==2.1.0 -f https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.1-cp310-cp310-linux_x86_64.whl
!pip -q install transformers==4.36.2 accelerate==0.25.0 sentencepiece

# 2) Imports and helper utilities
import os, time, gc, sys
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Import XLA only after installing the wheel
import torch_xla.core.xla_model as xm

def printt(*a, **k):
    """Print via xm.master_print if available, else normal print."""
    try:
        xm.master_print(" ".join(map(str,a)))
    except Exception:
        print(*a, **k)

# 3) Device detection: TPU -> CUDA -> CPU
def get_device():
    try:
        # will raise if TPU not initialized in this runtime
        xla_devices = xm.get_xla_supported_devices()
        device = xm.xla_device()
        backend = "TPU"
    except Exception as e:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        backend = "CUDA" if torch.cuda.is_available() else "CPU"
    printt(f"Backend: {backend} | Device: {device}")
    return backend, device

backend, device = get_device()

# 4) Model load routine with safe memory handling + fallback
PRIMARY_MODEL = "google/flan-t5-xl"   # recommended for TPU
FALLBACK_MODEL = "google/flan-t5-base"  # small fallback if memory fails

def try_load_model(model_name, device, use_bf16=True):
    """Try to load a T5 model. Returns (tokenizer, model)."""
    dtype = torch.bfloat16 if (use_bf16 and backend=="TPU") else None
    printt(f"Loading model {model_name} (dtype={dtype}) ...")
    tok = AutoTokenizer.from_pretrained(model_name)
    # AutoModelForSeq2SeqLM supports T5-style models
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_name,
        torch_dtype=dtype
    )
    # Move to device
    model.to(device)
    # On TPU, mark a step to sync XLA
    if backend == "TPU":
        xm.mark_step()
    printt("Loaded:", model_name)
    return tok, model

def safe_load(primary=PRIMARY_MODEL, fallback=FALLBACK_MODEL):
    tok = model = None
    try:
        tok, model = try_load_model(primary, device, use_bf16=True)
        return tok, model, primary
    except Exception as e:
        printt("⚠️ Primary load failed:", repr(e))
        # cleanup & collect
        try:
            del model
        except:
            pass
        gc.collect()
        if backend == "CUDA":
            try: torch.cuda.empty_cache()
            except: pass
        # Try fallback
        try:
            tok, model = try_load_model(fallback, device, use_bf16=False)
            return tok, model, fallback
        except Exception as e2:
            printt("⚠️ Fallback load also failed:", repr(e2))
            raise RuntimeError("Could not load either primary or fallback model.") from e2

tokenizer, model, active_model = safe_load()
printt("Active model in use:", active_model)

# 5) Generation helpers (T5 prompt + deterministic beam generation)
def t5_prompt(question, short=False):
    if short:
        return f"Answer in one word.\nQuestion: {question}\nAnswer:"
    return f"Question: {question}\nAnswer:"

def generate_answer(question, short=True, max_new_tokens=16, num_beams=4):
    prompt = t5_prompt(question, short=short)
    inputs = tokenizer(prompt, return_tensors="pt")
    # Move input tensors to correct device (XLA tensors)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    # Generation kwargs: deterministic beam search for factuality
    gen_kwargs = dict(max_new_tokens=max_new_tokens, do_sample=False, num_beams=num_beams)
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
    # mark XLA step if TPU
    if backend == "TPU":
        xm.mark_step()
    text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return text

# 6) Quick disk/memory status (optional, helpful)
def show_usage():
    try:
        printt("\n=== Disk usage ===")
        os.system("df -h / | sed -n '1,2p'")
    except Exception:
        pass

show_usage()

# 7) Example queries (edit these as you like)
questions = [
    "What is the capital of India?",
    "In one word: Who invented the telephone?",
    "Give a short summary of Einstein's theory of relativity in one sentence."
]

printt("\n--- Answers ---")
for q in questions:
    try:
        ans_short = generate_answer(q, short=True, max_new_tokens=8, num_beams=4)
        ans_full  = generate_answer(q, short=False, max_new_tokens=64, num_beams=4)
        printt(f"Q: {q}")
        printt("A (short):", ans_short)
        printt("A (full): ", ans_full)
        printt("-"*40)
    except Exception as ex:
        printt(f"Generation failed for question `{q}`:", repr(ex))

printt("\n✅ Done. Active model:", active_model)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.8 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement torch==2.1.0 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0)
ERROR: No matching distribution found for torch==2.1.0
ERROR: Could not find a version that satisfies the requirement torch_xla==2.1.0 (from versions: 2.8.0)
ERROR: No matching distribution found for torch_xla==2.1.0


/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/torch_xla/experimental/gru.py:113: SyntaxWarning: invalid escape sequence '\_'
  * **h_n**: tensor of shape :math:`(D * \text{num\_layers}, H_{out})` or
/tmp/ipython-input-1952578207.py:33: DeprecationWarning: Use torch_xla.device instead
  device = xm.xla_device()
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Backend: TPU | Device: xla:0
Loading model google/flan-t5-xl (dtype=torch.bfloat16) ...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


KeyboardInterrupt: 